In [7]:
from models.gap_mask import gap_mask
import os, sys
from keras.utils import multi_gpu_model as mgpu
import json
import numpy as np
from time import time

In [15]:
BPATH = sys.argv[2]
BPATH = '/scratch/ua349/fall2018/jobs/centered_0.json'

In [3]:
model = gap_mask()

 [!] Conv Spec:
 [!] 256 => 128: [64, 128]
 [!] 128 => 64: [256, 256, 256, 256]
 [!] 64 => 32: [256, 256, 256, 256]
 [!] 32 => 16: [512, 512, 512, 512]
 [!] 16 => 8: [512, 512, 512, 512]
 [!] 8 => 8: [512, 512, 512, 512]
 [!] 8 => 8: [1024, 1024, 2048, 2048]


In [4]:
model.load()

In [5]:
fastmodel = mgpu(model.coding, gpus=4)

In [6]:
with open(BPATH) as fl:
    spec = json.load(fl)
print(len(spec))

3555


In [14]:
tsize = 256
for ii, ent in enumerate(spec):
    t0 = time()
    fname = ent['from']
    dest = ent['to']
    
    
    fullres = np.load(fname).astype(np.float32) / 255**2
    batch = []
    for yy in range(0, 1024, tsize):
        for xx in range(0, 1024, tsize):
            batch.append(fullres[yy:yy+tsize, xx:xx+tsize])
    assert len(batch) == 16
#     print(np.max(batch))
#     break
    batch = np.array(batch).reshape((16, tsize, tsize, 1))
    
    canvas = np.zeros((4, 4, 2048))
    acts = fastmodel.predict(batch)
    
    sys.stdout.write('%d/%d %.2fs        \r' % (ii+1, len(spec), time() - t0))
    
    for yy in range(4):
        for xx in range(4):
            ind = yy*4+xx
            canvas[yy, xx] = acts[ind, :]
    
    np.save(dest, canvas)
    
#     break
            
    

KeyboardInterrupt: 